# Montly data of precip and temperature from MeteoSwiss

obained from http://www.meteoswiss.admin.ch/home/climate/past/homogenous-monthly-data.html?region=Table

In [ ]:
from cartopy.io import shapereader
import numpy as np
import geopandas
import matplotlib.pyplot as plt
import pandas as pd


import xarray as xr

In [ ]:
import io
import parse
import requests

%matplotlib inline

## Load lon/ lat coordinates of Switzerland

In [ ]:
# get natural earth data (http://www.naturalearthdata.com/)

# get country borders
resolution = '10m'
category = 'cultural'
name = 'admin_0_countries'

shpfilename = shapereader.natural_earth(resolution, category, name)

df = geopandas.read_file(shpfilename)

# read the swiss borders
poly = df.loc[df['ADMIN'] == 'Switzerland']['geometry']
ch = np.asarray(poly.values[0].exterior.coords)

lon = ch[:, 0]
lat = ch[:, 1]

plt.plot(lon, lat)

In [ ]:
# save to netcdf

ch = xr.Dataset(dict(lon=lon, lat=lat))

ch.attrs = dict(data='Outline of Switzerland in lon/ lat coordinates',
                source='http://www.naturalearthdata.com/')

ch.to_netcdf('../data/outline_switzerland.nc', format='NETCDF4_CLASSIC')

## Homogenized monthly station data

In [ ]:
def read_MCH_station_data(station):
    
    data = _get_data(station)
    
    station, lon, lat = _parse_header(data)
    
    df = _parse_data(data)
    
    return df, station, lon, lat
    
    
def _get_data(station):
    """ read data from web """
    
    url = ("http://www.meteoswiss.admin.ch/product/output/climate-data/"
           "homogenous-monthly-data-processing/data/homog_mo_{}.txt")
    
    full_url = url.format(station)
    
    # we need to pretend to be a browser
    headers = {'User-agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 '
                             '(KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'}
    
    # read txt file from web
    response = requests.get(full_url, headers=headers)
    
    return response.text
    
def _parse_header(data):
    """extract Station Name and Coordinates"""
    
    # loop through lines and extract Station Name and Coordinates
    for line in data.splitlines():
        
        if 'Station' in line:
            print(line)
            station = line.split('Station:')[1].strip()

        if "Coordinates" in line:
            # read lat and lon

            l = line.split('Coordinates:')[1].strip()
            
            r = parse.parse("{:g}° {:g}' N / {:g}° {:g}' E", l)
            
            lat = r[0] + r[1] / 60
            lon = r[2] + r[3] / 60

    
    return station, lon, lat    
    

def _parse_data(data):
    """read prec and temp into a pd.DataFrame"""
    
    # we need to pretend data is a string buffer
    df = pd.read_table(io.StringIO(data), header=18, delim_whitespace=True)

    # add day in order to parse the date
    df['day'] = 1

    date = pd.to_datetime(df[['Year', 'Month', 'day']])
    
    # only extract the necessary stuff   
    df = df[['Temperature', 'Precipitation']]
    
    # add date to df
    df.index = date
    return df
    
    
df, station, lon, lat = read_MCH_station_data('BAS')

In [ ]:

def to_netcdf(df, lon, lat, station, station_long):
    """save station data to netCDF file"""

    ds = df.to_xarray()
    
    ds = ds.rename(dict(index='time'))
    
    ds['lat'] = lat
    ds['lon'] = lon
    ds['station_long'] = station_long
    ds['station'] = station

    ds.attrs = dict(data='MeteoCH monthly station data',
                    source='http://www.meteoswiss.admin.ch/home/climate/past/homogenous-monthly-data.html')


    ds.Temperature.attrs = dict(unit='°C')
    ds.Precipitation.attrs = dict(unit='mm')

    
    ds.to_netcdf('../data/MCH_HOM_{}.nc'.format(station), format='NETCDF4_CLASSIC')
    
    return ds



In [ ]:
stations = ('BAS', 'BER', 'CHM', 'CHD', 'GSB', 'DAV', 'ENG', 'GVE', 'LUG', 'PAY', 'SIA', 'SIO', 'SAE', 'SMA')


LON = list()
LAT = list()

TEMP = list()
PREC = list()



for station in stations:
    df, station_long, lon, lat = read_MCH_station_data(station)

    # save as netcdf
    to_netcdf(df, lon, lat, station, station_long)
    
    
    LON.append(lon)
    LAT.append(lat)
    
    TEMP.append(df.Temperature.resample('A').mean().mean())
    
    # create annual sum of precipitation
    PREC.append(df.Precipitation.resample('A').sum().mean())
    
    
LON = np.asarray(LON)
LAT = np.asarray(LAT)

TEMP = np.asarray(TEMP)
PREC = np.asarray(PREC)

In [ ]:
# save climatology data to netcdf

number = range(len(stations))

data = dict(temp=('number', TEMP),
            prec=('number', PREC),
            lon=('number', LON),
            lat=('number', LAT),
            station=('number', list(stations)),)


ds = xr.Dataset(data_vars=data, coords=dict(number=('number', number)))


ds.attrs = dict(data='MeteoCH monthly station data, climatology',
                source='http://www.meteoswiss.admin.ch/home/climate/past/homogenous-monthly-data.html')


ds.temp.attrs['units'] = '°C'
ds.prec.attrs['units'] = 'mm / yr'


ds.to_netcdf('../data/MCH_clim.nc', format='NETCDF4_CLASSIC')

## Test scatter exercise

In [ ]:

mn = PREC.min()
mx = PREC.max()

p_scaled = ((PREC - mn) / (mx - mn)) * 200 + 50

p_scaled

In [ ]:

plt.plot(ch.lon, ch.lat)
h = plt.scatter(LON, LAT, c=TEMP, cmap='RdBu_r', vmax=8, vmin=-8, s=p_scaled, edgecolor='0.5')

plt.colorbar(h)

In [ ]:
import cartopy.crs as ccrs

ax = plt.axes(projection=ccrs.EuroPP())


ax.plot(ch.lon, ch.lat, transform=ccrs.PlateCarree())
h = ax.scatter(LON, LAT, c=TEMP, cmap='RdBu_r', vmax=8, vmin=-8, s=p_scaled, edgecolor='0.5',
               transform=ccrs.PlateCarree())

plt.colorbar(h)


In [ ]:

stations = ('BAS', 'BER', 'CHM', 'CHD', 'GSB', 'DAV', 'ENG', 'GVE', 'LUG', 'PAY', 'SIA', 'SIO', 'SAE', 'SMA')


MCH = dict()
MCH_anom = dict()

for station in stations:

    fN = '../data/MCH_HOM_{}.nc'.format(station)

    ds = xr.open_dataset(fN)
    
    MCH[station] = ds

    MCH_anom[station] = ds - ds.mean('time')

In [ ]:

for st in MCH_anom.values():

    st.Temperature.groupby('time.year').mean('time').plot.line('0.5')

In [ ]:
ds.mean('time')

In [ ]:
ds.Temperature.plot()